In [74]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
import joblib



In [75]:
data = pd.read_csv("/content/Housing.csv")
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [76]:
from re import L
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data['mainroad'] = le.fit_transform(data['mainroad'])
data['guestroom'] = le.fit_transform(data['guestroom'])
data['basement'] = le.fit_transform(data['basement'])
data['hotwaterheating'] = le.fit_transform(data['hotwaterheating'])
data['airconditioning'] = le.fit_transform(data['airconditioning'])
data['prefarea'] = le.fit_transform(data['prefarea'])
data['furnishingstatus'] = le.fit_transform(data['furnishingstatus'])
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,0


In [77]:
x = data.drop('price', axis = 1)
y = data['price']


In [78]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [79]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

In [80]:
x_train.shape

(436, 12)

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import RMSprop


In [82]:
model = Sequential()

In [83]:
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dense(18, activation ='swish' ))
model.add(Dense(26, activation ='swish' ))
model.add(Dense(20, activation ='swish' ))
model.add(Dense(15, activation ='swish' ))

model.add(Dense(1))

In [84]:
model.compile(optimizer = 'rmsprop', loss = 'mse')

In [85]:
history = model.fit(x_train, y_train, epochs = 200,batch_size=64, validation_split = 0.2)

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - loss: 24939061051392.0000 - val_loss: 24781600587776.0000
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 25414900645888.0000 - val_loss: 24781598490624.0000
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 25437786865664.0000 - val_loss: 24781594296320.0000
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 27102826987520.0000 - val_loss: 24781592199168.0000
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 25197606338560.0000 - val_loss: 24781585907712.0000
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 25402198196224.0000 - val_loss: 24781583810560.0000
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 25137072046080.0000 - val_loss: 24781573324800.0000
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 24759903453184.0000 - val_loss: 24781567033344.0000
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 26420298383360.0000 - val_loss: 24781556547584.0000
Epoch 10/2

In [86]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [87]:
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = root_mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = root_mean_squared_error(y_test, y_test_pred)

In [93]:
print(f'Train MSE: {train_mse}')
print(f'Train RMSE: {train_rmse}')
print(f'Test MSE: {test_mse}')
print(f'Test RMSE: {test_rmse}')

Train MSE: 22148024369152.0
Train RMSE: 4706169.0
Test MSE: 26551919837184.0
Test RMSE: 5152855.5


In [89]:
model.save('housing_model.h5')

In [90]:
from tensorflow.keras.models import load_model # Corrected the typo from 'tenserflow' to 'tensorflow'
loaded_model = load_model('/content/housing_model.h5',custom_objects={'mse': mean_squared_error})

In [91]:
new_data = pd.DataFrame([{
    'area': 1500,
    'bedrooms': 3,
    'bathrooms': 2,
    'stories': 2,
    'mainroad': 1,
    'guestroom': 1,
    'basement': 1,
    'hotwaterheating': 1,
    'airconditioning': 1,
    'parking': 2,
    'prefarea': 1,
    'furnishingstatus': 1
}])

In [92]:
new_data_scaled = scaler.transform(new_data)
prediction = model.predict(new_data_scaled)
print(f'Predicted Price: {prediction[0][0]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted Price: 768140.5625
